<a href="https://colab.research.google.com/github/mamba124/just_for_fun/blob/classifier-in-autoenc/Denoising_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Из кераса загружаем необходимые слои для нейросети
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K # подтягиваем базовые керасовские функции
from tensorflow.keras.optimizers import Adam # загружаем выбранный оптимизатор
from tensorflow.keras import utils # загружаем утилиты кераса
from google.colab import files # модуль для загрузки файлов в colab
import matplotlib.pyplot as plt # из библиотеки для визуализации данных возьмём интерфейс для построения графиков простых функций
from tensorflow.keras.preprocessing import image # модуль для отрисовки изображения
import numpy as np # библиотека для работы с массивами данных
import librosa #Параметризация аудио
import os 

import pandas as pd # библиотека для анализа и обработки данных
from PIL import Image # модуль для отрисовки изображения
from sklearn.model_selection import train_test_split # модуль для разбивки выборки на тренировочную/тестовую
from sklearn.preprocessing import StandardScaler # модуль для стандартизации данных

from tensorflow.keras.models import Model, load_model #из кераса подгружаем абстрактный класс базовой модели, метод загрузки предобученной модели
from tensorflow.keras.optimizers import RMSprop #из кераса загружаем выбранный оптимизатор

#Автокодировщик

In [0]:
def baseAutoencoder(): # зададим функцию создания базового автокодировщика
    img_input = Input((28,28,1)) # задаём входные размеры

    x = Conv2D(32, (3, 3), padding='same', activation='relu')(img_input) # входные данные передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # затем пропускаем через слой нормализации данных 
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # далее снова слой двумерной свёртки
    x = BatchNormalization()(x) # и еще слой нормализации данных
    x = MaxPooling2D()(x) # передаём на слой подвыборки, снижающий размерность поступивших на него данных

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # пропускаем через слой нормализации данных 
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)  # далее снова слой двумерной свёртки
    x = BatchNormalization()(x) # и еще слой нормализации данных
    x = MaxPooling2D()(x) # передаём на слой подвыборки
    # Изображение ужали до 7*7

    x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', activation='relu')(x) # слой разжимает данные(с 7*7 на 14*14)
    x = BatchNormalization()(x) # слой нормализации данных
    
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # ещё слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    # Сжатие MaxPooling2D не применяем

    x = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', activation='relu')(x) # слой разжимает данные(с 14*14 на 28*28)
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # ещё слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных

    # Финальный слой двумерной свертки, выдающий итоговое изображение
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) # указываем модель, с оригинальным изображением на входе в сеть и сжатым-разжатым на выходе из сети
    model.compile(optimizer=Adam(),
                  loss='mean_squared_error') # компилируем модель с оптимайзером Адам и среднеквадратичной ошибкой

    return model # функция вернёт заданную модель

# AUDIO

In [3]:
!pip install soundfile

In [0]:
import librosa # for mel-spectrogram estimation
import soundfile # for opening .flac audio
from matplotlib import pyplot as plt
import numpy as np

In [0]:
st= '/content/drive/My Drive/train/clean/47/47_122796_47-122796-0071 (1).npy'
stop = st[-7:]

In [0]:
def get_chunks(signal, length, step):
  batch =[]
  syms_len = len(signal)
  index = 0
  
  while (index + length <= syms_len):
    batch.append(signal[index:index+length])
    index += step

  return batch

def prepare_chunks(audio,length, step):
  batch_list = []
  for item in audio:
    batch_list.append(get_chunks(item, length, step))

  return np.array(batch_list)

In [11]:
directory = '/content/drive/My Drive/train/'
folders = os.listdir(directory)
clean_arrays = []
noisy_arrays = []
patch = os.listdir(directory+'clean/')

for folder in patch[:200]:
  for arr in os.listdir(directory+'clean/'+folder):
    if arr[-7:]!=stop:
      clean_arrays.append(np.load(directory+'clean/'+folder+'/'+arr))
      noisy_arrays.append(np.load(directory+'noisy/'+folder+'/'+arr))



KeyboardInterrupt: ignored

In [0]:
clean_arrays_ = []
noisy_arrays_ = []

for i in range(len(clean_arrays[:-1])):
  for j in range(len(clean_arrays[i])):
    clean_arrays_.append(clean_arrays[i][j])
    noisy_arrays_.append(noisy_arrays[i][j])

In [0]:
length = 60 
step = 50

x_trainC = prepare_chunks(clean_arrays_, length, step)
y_trainC = np.ones(x_trainC.shape)

x_trainN = prepare_chunks(noisy_arrays_, length, step)
y_trainN = np.zeros(x_trainN.shape)

In [0]:
def classificator(vector):
  shape_ = (vector.shape[1],vector.shape[2])
  inputs = layers.Input(shape = shape_)
  hidden = layers.Conv1D(8, 3)(inputs)
  hidden = layers.Activation('tanh')(hidden)

  hidden = layers.Flatten()(hidden)
  hidden = layers.Dense(56)(hidden)
  hidden = layers.Activation('relu')(hidden)

  hidden = layers.Dense(100)(hidden)
  out = layers.Dense(1, activation = 'sigmoid')(hidden)
  
  model = Model(inputs = inputs, outputs = out)

  return model

def autoencoder(vector):
  #result = model.predict(vector)
  inputs_decision = layers.Input(shape = (1,))
  inputs_vector = layers.Input(shape = (vector.shape[1],vector.shape[2]))
  ##Block1##
  hidden = layers.Conv1D(16, 3) (inputs_vector)
  hidden = layers.Activation('tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(16, 3)(hidden)
  hidden = layers.Activation('relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(hidden)

  out_class = layers.Flatten()(hidden)

  out_class = layers.Dense(56, activation = 'relu')(out_class)
  out_class = layers.Dense(1, activation = 'sigmoid')(out_class)

  ##Block2##
  hidden = layers.Conv1D(64, 3) (hidden)
  hidden = layers.Activation('tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(64, 3)(hidden)
  hidden = layers.Activation('relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(hidden)

  
  ##Block3##
  #hidden = Conv1D(128, 3) (hidden)
  #hidden = Activation('tanh')
  #hidden = BatchNormalization()(hidden)
  #hidden = Conv1D(128, 3)(hidden)
  #hidden = Activation('relu')
  #hidden = BatchNormalization()(hidden)
  #hidden = MaxPooling1D(hidden)  

  ##UPBlock1##
  hidden = layers.UpSampling1D(hidden)
  hidden = layers.Conv1D(64, 3, activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  
  ##UPblock2
  hidden = layers.UpSampling1D(hidden)
  hidden = layers.Conv1D(16, 3, activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  hidden = layers.Flatten()(hidden)
  hidden = layers.Concatenate([out_class, hidden])
  hidden = layers.Dense(128, activation='relu')(hidden)
  out = layers.Dense(inputs_vector.size)(hidden)

  modelD = Model(inputs=inputs_vector, outputs = [out, out_class])

  return modelD

  

In [17]:
def shuffle(a, b):
    p = np.random.permutation(len(a)) 
    return a[p], b[p] 
x_train = np.concatenate(x_trainC, x_trainN)
y_train = np.concatenate(y_trainC, y_trainN)

x_train_, y_train_ = shuffle(x_train, y_train)
#x_val_, y_val_ = shuffle(x_val, y_val)

NameError: ignored

In [30]:
classificator = classificator(clean_arrays)
classificator.compile(optimizer = RMSprop(), loss = 'mse', metrics = 'accuracy')
classificator.fit(x_train_, y_train_, batch_size=100, epochs=20, validation_data=x_val_, y_val_)

1/1 [==============================] - 0s 1ms/step - loss: 0.2353 - accuracy: 1.0000


In [0]:
denoise = autoencoder(clean_arrays_)
denoise.compile(optimizer = RMSprop(), loss = 'mse', metrics = 'accuracy')
denoise.fit(x_train_, [x_train_, y_train_])

In [0]:
denoised, classes = denoise.predict(x_train_[-300:])

In [0]:
def glue_chunks(signal_list):
  signals = []
  for signal in signal_list:
    glued = []
    for i in range(len(signal)):
      if i == 0: ind = 0
      else: ind = length - step
      for j in range(ind,length):
        glued.append(signal[i][j])
    signals.append(np.array(glued))

  signals = np.array(signals)

denoised_glued = glue_chunks(denoised)

In [0]:
for i, record in enumerate(denoised_glued): 
  np.save('/final/' + i, record)